In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [4]:
##load the ann trained model, scalar pickle file, ohe pickle file then predicting
from tensorflow.keras.models import load_model
model=load_model('model.h5')


##load the encoder and scaler pickle files
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [5]:
#Example input data for prediction
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

Converting categorical values(geo,gender) to numerical and concatinating to og df

In [ ]:

type(onehot_encoder_geo)


sklearn.preprocessing._encoders.OneHotEncoder

In [ ]:
input_data = pd.DataFrame([input_data]) #already executed once

ValueError: Must pass 2-d input. shape=(1, 1, 10)

In [17]:
type(input_data[['Geography']])

pandas.core.frame.DataFrame

In [19]:
geo_encoded = onehot_encoder_geo.transform(input_data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [21]:
#combine ohe col with eg input data
input_data=pd.concat([input_data.reset_index(drop=True), geo_encoded_df],axis=1)
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0,1.0,0.0,0.0


In [23]:
#encode categorical var
input_data['Gender']=label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
 #appending/concatenating changes in geography column to og dataframe, already executed once
input_data=pd.concat([input_data.drop("Geography",axis=1),geo_encoded_df],axis=1)


KeyError: "['Geography'] not found in axis"

In [28]:
input_data = input_data.loc[:, ~input_data.columns.duplicated()]

In [29]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


Scaling the input data

In [30]:
input_scaled=scaler.transform(input_data)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [31]:
#predict churn(whether person will leave teh bank or not)
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 186ms/step


array([[0.02463349]], dtype=float32)

In [32]:
prediction_proba = prediction[0][0]

In [33]:
prediction_proba

0.024633486

In [34]:

if prediction_proba >= 0.5:
    print("Customer will churn")
else:
    print("Customer will NOT churn")

Customer will NOT churn
